In [42]:
from osgeo import gdal
from osgeo import ogr
from osgeo import osr
import numpy as np 
import pandas as pd 

In [52]:
### read tif and save transformed data
# hour = 1
firescen = 3
all_points = []
for hour in range(1,11):
    gdal.Warp("flame_length_4326/firescen{}_hr{}_4326.TIF".format(firescen, hour), 
              gdal.Open('flame_length_scen{}/{}.TIF'.format(firescen, hour)), 
              dstSRS='EPSG:4326')
    src = gdal.Open("flame_length_4326/firescen{}_hr{}_4326.TIF".format(firescen, hour))
    ulx, xres, xskew, uly, yskew, yres  = src.GetGeoTransform()
    lrx = ulx + (src.RasterXSize * xres)
    lry = uly + (src.RasterYSize * yres)

    def pixel2coord(col, row):
        """Returns global coordinates to pixel center using base-0 raster index"""
        c, a, b, f, d, e = ulx, xres, xskew, uly, yskew, yres
        xp = a * col + b * row + a * 0.5 + b * 0.5 + c
        yp = d * col + e * row + d * 0.5 + e * 0.5 + f
        return xp, yp

    val, x_loc, y_loc = [], [], []
    raster_array = src.GetRasterBand(1).ReadAsArray()
    for x in range(src.RasterXSize):
        for y in range(src.RasterYSize):
            if raster_array[y, x]>0:
                x_loc.append(x)
                y_loc.append(y)
                val.append(raster_array[y, x])
    print(len(val))
    x_coord, y_coord = pixel2coord(np.array(x_loc), np.array(y_loc))
    point_df = pd.DataFrame({'lon': x_coord, 'lat': y_coord, 'flame_length': val})
    point_df['t_hour'] = hour
    all_points.append(point_df)
#     point_df.to_csv('flame_length_point_4326/point_{}hr.csv'.format(hour))

all_points_df = pd.concat(all_points).to_csv('flame_length_point_4326/point_firescen{}.csv'.format(firescen), index=False)

237
705
1442
2017
2738
3427
3928
4306
4612
4946
